In [4]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np
import math

In [ ]:
## Steps for Git
Git add <file> 
Git commit -m "comments" 
Git push origin master

In [63]:
## Initizlization

#Create SQLite
conn = sqlite3.connect('praiselyrics.db')
# df = pd.read_sql_query("select * from lyrics_url;",conn)

In [6]:
session = requests.session()
session.proxies = {'https':'socks5://127.0.0.1:9050','http':'socks5://127.0.0.1:9050'}

In [55]:
user_agent_list = ['Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
                   'Mozilla/5.0 (X11; Ubuntu; Linux armv7l; rv:17.0) Gecko/20100101 Firefox/17.0'
                  ]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

base_url = "http://worship-songs-resources.worshiptogether.com/nav/category/songs/"

columns = ['url','title','status']
dflyrics = pd.DataFrame(columns=columns)

# Max = 158
# 158 (includ) onwards
for i in range(0,157):
    try: 
        url = base_url+str(i*12)
        # print(url)
        pause = randint(3,5)
        time.sleep(pause)
        response = session.get(url, headers=headers)
        data = response.text
        song_url = BeautifulSoup(data,"lxml")
        song_url_list = song_url.findAll('div',attrs={'class','post_result'})
        for s in song_url_list:
            a = s.find('h2').find('a')
            url = a.get('href')
            title = a.get_text()
            status = 0 
            row = [url,title,status]
            dflyrics.loc[len(dflyrics)] = row  
    except Exception as e:
        print(url)
        print(e)

dflyrics.to_sql('lyrics_url',conn,if_exists='replace')

In [58]:

dflyrics.reset_index(inplace=True)
dflyrics['key'] = dflyrics['index']
del dflyrics['index']
#alter table lyrics_url add column fullhtml text;

dflyrics.to_sql('lyrics_url',conn,if_exists='replace')

In [62]:
def set_new_ip():
    """Change IP using TOR"""
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

cursor = conn.cursor()
        
batch = 1000
mini_batch = 100 
lower_limit = 3
upper_limit = 5

iterations = 0
count_query = pd.read_sql_query("select count(*) as cnt from lyrics_url where status = 0;",conn)
count = int(count_query['cnt'][0])
# initial_count = count - batch
initial_count = 1700 

while count > initial_count : 

    count_query = pd.read_sql_query("select count(*) as cnt from lyrics_url where status = 0;",conn)
    count = int(count_query['cnt'][0])

    # Randomize 
    set_new_ip()
    rand_user_agent = randint(0,len(user_agent_list)-1)
    headers['User-Agent'] = user_agent_list[rand_user_agent]
    elapsed_time = 0 
    # Scrap
    scrap = pd.read_sql_query("select key,url from lyrics_url where status = 0 limit "+str(mini_batch)+";",conn)
    t0 = time.time()
    for key,row in scrap.iterrows():

        pause = randint(lower_limit,upper_limit)
        time.sleep(pause)

        key = row['key']
        url = row['url']
        response = session.get(url, headers=headers)
        elapsed_time += response.elapsed.total_seconds()
        data = response.text

        
            
        cursor.execute("""UPDATE lyrics_url SET fullhtml = ?, status = ? WHERE key = ? """,
              (data,1,key))
        conn.commit()
            
    t1 = time.time()
    print("Interations :",iterations," Per Request:" ,elapsed_time/mini_batch," Total:",t1-t0)
    # print("Initial_count ",initial_count," Current:",count)
    iterations = iterations + 1
    

cursor.close()

Interations : 0  Per Request: 0.9110592000000001  Total: 583.5309026241302
Interations : 1  Per Request: 0.9240130199999999  Total: 571.5068211555481
Interations : 2  Per Request: 0.9276187900000002  Total: 582.3232312202454


In [65]:
count_query = pd.read_sql_query("select count(*) as cnt from lyrics_url where status = 0;",conn)
count_query

,cnt
0,1578


In [ ]:
# Maranatha Music / Integrity Music / Vineyard Music / Passion

In [8]:
#Page Extract
#http://www.worshiptogether.com/songs/holy-spirit/

base_url = "http://www.worshiptogether.com/songs/holy-spirit/"
response = session.get(base_url, headers=headers)
data = response.text
soup = BeautifulSoup(data,"lxml")
# p = soup.find('div',attrs={'class':'sub_navi'})
# no_records = re.findall(".*of (\d*) artists.*",p.get_text())
# page = math.ceil(int(no_records[0]) / 75)
# print(url_1 ," ", response.elapsed.total_seconds()," ",page)



In [13]:
lyrics = soup.find('div',attrs={'class':'chord-pro-disp'})
lyrics.get_text()

"\n\n\n\nIntro\n\n\n\n\nA /// //// D /// ////\n\n\n\n\n\nA /// //// D /// ////\n\n\n\n\n\n\n\nVerse 1\n\n\n\n\nA\n  There's nothing worth more, that will ever come \n\n\nD\nclose\n\n\n\n\n\nNo thing can compare, You're our living \n\n\nA\nhope\n\n\n\n\n\nYour Presence \n\n\nD\nLord\n\n\n\n\n\n\nVerse 2\n\n\n\n\nA\n  I've tasted and seen, of the sweetest of \n\n\nD\nloves\n\n\n\n\n\nWhere my heart becomes free, and my shame is und\n\n\nA\none\n\n\n\n\n\nIn Your Presence \n\n\nD\nLord\n\n\n\n\n\n\nChorus \n\n\n\n\nA\nHoly Spirit You are welcome here\n\n\n\n\n\nCome \n\n\nD\nflood this place and fill the \n\n\nBm\natmosphere\n\n\n\n\n\nYour \n\n\nA\nGlory God is what our hearts long for\n\n\n\n\n\nTo be \n\n\nD\novercome by Your \n\n\nBm\nPresence Lord\n\n\n\n\n\n\nInterlude\n\n\n\n\nA /// //// D /// Bm ///\n\n\n\n\n\n\n\nREPEAT VERSE 1\n\n\n\n\n\nREPEAT VERSE 2\n\n\n\n\n\nREPEAT CHORUS 2X\n\n\n\n\n\n\nInterlude\n\n\n\n\nA /// //// D /// Bm ///\n\n\n\n\n\nA /// //// D /// Bm ///\n\n\n\n\n

In [20]:
getLyrics = soup.findAll('div',attrs={'class':'chord-pro-lyric'})
for i in getLyrics:
    print(i.text)
    #print(i.get_text().replace('\n',''))

Intro


Verse 1
  There's nothing worth more, that will ever come 
close
No thing can compare, You're our living 
hope
Your Presence 
Lord
Verse 2
  I've tasted and seen, of the sweetest of 
loves
Where my heart becomes free, and my shame is und
one
In Your Presence 
Lord
Chorus 
Holy Spirit You are welcome here
Come 
flood this place and fill the 
atmosphere
Your 
Glory God is what our hearts long for
To be 
overcome by Your 
Presence Lord
Interlude

REPEAT VERSE 1
REPEAT VERSE 2
REPEAT CHORUS 2X
Interlude


Bridge
Let us become more aware of Your Presence
Let us experience the Glory of Your Goodness
  Let us be
come more a
ware of Your 
Presence
  Let us exp
erience the 
Glory of Your 
Goodness
 X3 
Lord
REPEAT CHORUS DOWN


In [26]:
getTaxonomy = soup.findAll('div',attrs={'class':'song_taxonomy'})
for tax in getTaxonomy:
    print(tax.get_text())




Writer(s):               
Katie Torwalt, Bryan Torwalt                            




                                    Theme(s):
                                
Call to Worship                            



Tempo:
Slow




Ministry(s):
Jesus Culture                            



BPM:
                                72
                            



Original Key(s):
A 



Key Range:
                                G-C
                            



Recommended Key(s):
A , D , D#/Eb , G 



CCLI #:
                                6087919
                            



Scripture Reference(s):
                                2 Corinthians 3:17; Psalm 96: 8-13
                            


